# generating HiC heatmap on the whole genome

* using Juicebox to generate the graph for the publication
* instructions: https://github.com/phasegenomics/juicebox_scripts

## 1. generating mapping bam file

In [33]:
REF=/output/genomic/plant/Vaccinium/myrtillus/Genome/Assembly/bilberry_chr01-12_plusUnanchoredSeqs.NCBI.fasta

In [34]:
HICR1_RAW=/workspace/hraczw/github/GA/Bilberry_genome/002.Fastp.trimming/R1.cleaned.specifiedAdapter.short.Q15.fq.gz
HICR2_RAW=/workspace/hraczw/github/GA/Bilberry_genome/002.Fastp.trimming/R2.cleaned.specifiedAdapter.short.Q15.fq.gz

In [3]:
module load ncbi-blast
module list

Currently Loaded Modulefiles:
  1) powerPlant/core    4) git/2.21.0         7) asub/2.2
  2) texlive/20151117   5) perlbrew/0.76      8) ncbi-blast/2.6.0
  3) pandoc/1.19.2      6) perl/5.28.0


In [4]:
mkdir 022.JuiceBox_visualisation

In [4]:
WDR=022.JuiceBox_visualisation

In [ ]:
# make a list of chrs i want to extract into a file called chrs.txt under $WDR
Chr01
Chr02
Chr03
Chr04
Chr05
Chr06
Chr07
Chr08
Chr09
Chr10
Chr11
Chr12

In [9]:
makeblastdb -in $REF -dbtype nucl -parse_seqids -out $WDR/bilberryGenome



Building a new DB, current time: 01/29/2021 11:21:19
New DB name:   /powerplant/workspace/hraczw/github/GA/Bilberry_genome/022.JuiceBox_visualisation/bilberryGenome
New DB title:  /output/genomic/plant/Vaccinium/myrtillus/Genome/Assembly/bilberry_chr01-12_plusUnanchoredSeqs.NCBI.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1418 sequences in 7.87916 seconds.


In [11]:
blastdbcmd -db $WDR/bilberryGenome -dbtype nucl -entry_batch $WDR/chrs.txt -out $WDR/BilberryGenome_chrs.fasta

In [12]:
grep '>' $WDR/BilberryGenome_chrs.fasta

>Chr01 
>Chr02 
>Chr03 
>Chr04 
>Chr05 
>Chr06 
>Chr07 
>Chr08 
>Chr09 
>Chr10 
>Chr11 
>Chr12 


In [14]:
REF_CHRS=$WDR/BilberryGenome_chrs.fasta

In [37]:
# mapping following in steps on ALLHIC wiki

module load bwa
module list

Currently Loaded Modulefiles:
  1) powerPlant/core    5) perlbrew/0.76      9) bedtools/2.27.1
  2) texlive/20151117   6) perl/5.28.0       10) bwa/0.7.17
  3) pandoc/1.19.2      7) asub/2.2
  4) git/2.21.0         8) samtools/1.9


In [15]:
bsub -J bwa \
-o $WDR/bwa_index.out \
-e $WDR/bwa_index.err \
"bwa index $REF_CHRS"

Job <886530> is submitted to default queue <lowpriority>.


In [38]:
module load samtools
module list

Currently Loaded Modulefiles:
  1) powerPlant/core    5) perlbrew/0.76      9) bedtools/2.27.1
  2) texlive/20151117   6) perl/5.28.0       10) bwa/0.7.17
  3) pandoc/1.19.2      7) asub/2.2
  4) git/2.21.0         8) samtools/1.9


In [39]:
bsub -J map \
-m wkoppg31 \
-n 40 \
-o $WDR/map_pg.out \
-e $WDR/map_pg.err \
"bwa mem -5SP \
-t 40 \
$REF_CHRS \
$HICR1_RAW \
$HICR2_RAW | \
samtools view -S -h -b -F 2316 > $WDR/mapped_pg.bam"

Job <887510> is submitted to default queue <lowpriority>.


## 2. generating .hic file for juicerbox

* using phasegenomics scripts: https://github.com/phasegenomics/juicebox_scripts

In [23]:
python /workspace/hraczw/github/programs/juicebox_scripts/juicebox_scripts/makeAgpFromFasta.py $REF_CHRS $WDR/bilberry_chrs.agp

In [31]:
python /workspace/hraczw/github/programs/juicebox_scripts/juicebox_scripts/agp2assembly.py $WDR/bilberry_chrs.agp $WDR/bilberry_chrs.assembly

In [1]:
export PATH=/workspace/hraczw/github/programs/matlock/bin/:$PATH

In [5]:
bsub -J convert -m wkoppb50 -o $WDR/matlock_convert.out -e $WDR/matlock_convert.err \
"matlock bam2 juicer $WDR/mapped_pg.bam $WDR/out.links.txt"

Job <887654> is submitted to default queue <lowpriority>.


In [6]:
bsub -J sort -o $WDR/sort.out -e $WDR/sort.err \
"sort -k2,2 -k6,6 $WDR/out.links.txt > $WDR/out.sorted.links.txt"

Job <887665> is submitted to default queue <lowpriority>.


In [7]:
bsub -J convertToHic -o $WDR/convertToHic.out -e $WDR/convertToHic.err \
"bash /workspace/hraczw/github/programs/3d-dna/visualize/run-assembly-visualizer.sh -p false $WDR/bilberry_chrs.assembly $WDR/out.sorted.links.txt"

Job <887666> is submitted to default queue <lowpriority>.
